In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import requests\n",
    "from bs4 import BeautifulSoup\n",
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "<ipython-input-4-321e5b2648a8>:16: SettingWithCopyWarning: \n",
      "A value is trying to be set on a copy of a slice from a DataFrame.\n",
      "Try using .loc[row_indexer,col_indexer] = value instead\n",
      "\n",
      "See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy\n",
      "  games_final['Day'] = pd.to_datetime(games_final['Day'])\n"
     ]
    }
   ],
   "source": [
    "#Getting dates and gameid for each game in 2020/2021 English Premier League\n",
    "\n",
    "url='https://api.sportsdata.io/v3/soccer/scores/json/CompetitionDetails/EPL'\n",
    "\n",
    "response=requests.get(url, headers = {\"Ocp-Apim-Subscription-Key\": \"7e69e12a9f714caebb8c178cf904e445\"})\n",
    "\n",
    "result=pd.json_normalize(response.json())\n",
    "games = pd.DataFrame(result['Games'][0])#.flatten()#.columns\n",
    "\n",
    "games_final = games[['GameId','AwayTeamId','HomeTeamId','Day','AwayTeamName','HomeTeamName']]\n",
    "\n",
    "Days=games_final['Day'].unique().tolist()\n",
    "\n",
    "Days=[element[0:10] for element in Days]\n",
    "\n",
    "games_final['Day'] = pd.to_datetime(games_final['Day'])\n",
    "\n",
    "TeamID = games_final['AwayTeamId'].unique().tolist()\n",
    "\n",
    "Days=Days[:18]  ##### IMPORTANT: FOR DATES BEYOND THIS, EVENTS STILL HAVEN'T HAPPEN, SO CODE DOESN'T WORK\n",
    "\n",
    "#Function that returns a dataframe of the games for each date\n",
    "\n",
    "def daily_games(date):\n",
    "    \n",
    "    url='https://api.sportsdata.io/v3/soccer/odds/json/GameOddsByDate/'+date\n",
    "\n",
    "    response=requests.get(url, headers = {\"Ocp-Apim-Subscription-Key\": \"7e69e12a9f714caebb8c178cf904e445\"})\n",
    "\n",
    "    response=response.json()\n",
    "    result=pd.DataFrame(response)\n",
    "\n",
    "    #Selecting games only from EPL\n",
    "\n",
    "    result=result[result['RoundId']==602].reset_index()\n",
    "\n",
    "    \n",
    "    #Selecting games which happened\n",
    "\n",
    "    result=result[result['Status']=='Final']\n",
    "\n",
    "    #Aggregated Odds per game \n",
    "\n",
    "    odds=result['PregameOdds']\n",
    "\n",
    "    empty_list = []\n",
    "\n",
    "    for i in range(len(odds)):\n",
    "    \n",
    "        try:\n",
    "        \n",
    "            empty_list.append(pd.DataFrame(odds[i]))\n",
    "    \n",
    "        except: \n",
    "        \n",
    "            continue\n",
    "        \n",
    "    games_odds=pd.concat(empty_list)\n",
    "\n",
    "    games=games_odds.groupby('GameId').agg({'HomeMoneyLine':'mean','AwayMoneyLine':'mean','DrawMoneyLine':'mean'})\n",
    "\n",
    "    #Merging with initial table\n",
    "\n",
    "    daily_games = pd.merge(left = result,\n",
    "                                 right = games,\n",
    "                                 how = 'inner', \n",
    "                                 left_on = \"GameId\", \n",
    "                                 right_on= \"GameId\")\n",
    "\n",
    "\n",
    "    daily_games=daily_games[['GameId', 'RoundId','Day', \n",
    "                         'AwayTeamId', 'HomeTeamId', 'AwayTeamName', 'HomeTeamName','HomeTeamScore',\n",
    "                         'AwayTeamScore', 'TotalScore','HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine']]\n",
    "\n",
    "    daily_games[['Day']]=pd.to_datetime(daily_games['Day'])\n",
    "    \n",
    "    return daily_games\n",
    "\n",
    "#Applying above function to all dates contained in the list previously obtained. \n",
    "#The result is a dataframe with all games, scores and bets for the 2020/2021 EPL\n",
    "\n",
    "final_result=pd.concat(list(map(daily_games,Days)))\n",
    "\n",
    "#Building a column for the actual winner, based on scores presented\n",
    "\n",
    "def winner(row):\n",
    "    if row['HomeTeamScore']>row['AwayTeamScore']:\n",
    "        \n",
    "        return row['HomeTeamName']\n",
    "    \n",
    "    elif row['HomeTeamScore']<row['AwayTeamScore']:\n",
    "        \n",
    "        return row['AwayTeamName']\n",
    "    \n",
    "    elif row['HomeTeamScore']==row['AwayTeamScore']:\n",
    "        \n",
    "        return 'Tie'\n",
    "\n",
    "final_result['Winner']=final_result.apply(winner,axis=1)\n",
    "\n",
    "#Building a column for the predicted winner, based on bets presented\n",
    "\n",
    "def moneyline(row):\n",
    "    minimum=min(row['HomeMoneyLine'], row['AwayMoneyLine'], row['DrawMoneyLine'])\n",
    "    if row['HomeMoneyLine']==minimum:\n",
    "        return row['HomeTeamName']\n",
    "    elif row['AwayMoneyLine']==minimum:\n",
    "        return row['AwayTeamName']\n",
    "    elif row['DrawMoneyLine']==minimum:\n",
    "        return 'Tie'\n",
    "    \n",
    "final_result['Moneyline']=final_result.apply(moneyline,axis=1)\n",
    "\n",
    "#Rearraging the dataframe \n",
    "\n",
    "final_result.reset_index(inplace=True)\n",
    "\n",
    "final_result.drop(['index'],axis=1)\n",
    "\n",
    "final_result=final_result[['Day','HomeTeamName','AwayTeamName','Winner',\n",
    "       'Moneyline']]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Day</th>\n",
       "      <th>HomeTeamName</th>\n",
       "      <th>AwayTeamName</th>\n",
       "      <th>Winner</th>\n",
       "      <th>Moneyline</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>2020-09-12</td>\n",
       "      <td>Fulham FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>2020-09-12</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>2020-09-13</td>\n",
       "      <td>West Bromwich Albion FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>2020-09-13</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>2020-09-14</td>\n",
       "      <td>Sheffield United FC</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>2020-09-14</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Chelsea FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>2020-09-19</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>West Bromwich Albion FC</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Everton FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>2020-09-19</td>\n",
       "      <td>Leeds United FC</td>\n",
       "      <td>Fulham FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Leeds United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>2020-09-19</td>\n",
       "      <td>Manchester United FC</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>2020-09-19</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>West Ham United FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Arsenal FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>10</th>\n",
       "      <td>2020-09-20</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>11</th>\n",
       "      <td>2020-09-20</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>12</th>\n",
       "      <td>2020-09-20</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>13</th>\n",
       "      <td>2020-09-20</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Burnley FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>14</th>\n",
       "      <td>2020-09-21</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Sheffield United FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>15</th>\n",
       "      <td>2020-09-21</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>16</th>\n",
       "      <td>2020-09-26</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>17</th>\n",
       "      <td>2020-09-26</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Everton FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18</th>\n",
       "      <td>2020-09-26</td>\n",
       "      <td>West Bromwich Albion FC</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Chelsea FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>19</th>\n",
       "      <td>2020-09-26</td>\n",
       "      <td>Burnley FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>20</th>\n",
       "      <td>2020-09-27</td>\n",
       "      <td>Sheffield United FC</td>\n",
       "      <td>Leeds United FC</td>\n",
       "      <td>Leeds United FC</td>\n",
       "      <td>Leeds United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>21</th>\n",
       "      <td>2020-09-27</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>22</th>\n",
       "      <td>2020-09-27</td>\n",
       "      <td>Manchester City FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>23</th>\n",
       "      <td>2020-09-27</td>\n",
       "      <td>West Ham United FC</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>West Ham United FC</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24</th>\n",
       "      <td>2020-09-28</td>\n",
       "      <td>Fulham FC</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>25</th>\n",
       "      <td>2020-09-28</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>26</th>\n",
       "      <td>2020-10-03</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Chelsea FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>27</th>\n",
       "      <td>2020-10-03</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Everton FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>28</th>\n",
       "      <td>2020-10-03</td>\n",
       "      <td>Leeds United FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Manchester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>29</th>\n",
       "      <td>2020-10-03</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "      <td>Burnley FC</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>30</th>\n",
       "      <td>2020-10-04</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>West Ham United FC</td>\n",
       "      <td>West Ham United FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>31</th>\n",
       "      <td>2020-10-04</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>West Bromwich Albion FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>32</th>\n",
       "      <td>2020-10-04</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>Fulham FC</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>2020-10-04</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>Sheffield United FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>34</th>\n",
       "      <td>2020-10-04</td>\n",
       "      <td>Manchester United FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>35</th>\n",
       "      <td>2020-10-04</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>36</th>\n",
       "      <td>2020-10-17</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Liverpool FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>37</th>\n",
       "      <td>2020-10-17</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Chelsea FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>38</th>\n",
       "      <td>2020-10-17</td>\n",
       "      <td>Manchester City FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>39</th>\n",
       "      <td>2020-10-17</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "      <td>Manchester United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>40</th>\n",
       "      <td>2020-10-18</td>\n",
       "      <td>Sheffield United FC</td>\n",
       "      <td>Fulham FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Sheffield United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>41</th>\n",
       "      <td>2020-10-18</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>42</th>\n",
       "      <td>2020-10-18</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>West Ham United FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>43</th>\n",
       "      <td>2020-10-18</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>44</th>\n",
       "      <td>2020-10-19</td>\n",
       "      <td>West Bromwich Albion FC</td>\n",
       "      <td>Burnley FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>West Bromwich Albion FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>45</th>\n",
       "      <td>2020-10-19</td>\n",
       "      <td>Leeds United FC</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>Leeds United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>46</th>\n",
       "      <td>2020-10-23</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "      <td>Leeds United FC</td>\n",
       "      <td>Leeds United FC</td>\n",
       "      <td>Aston Villa FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>47</th>\n",
       "      <td>2020-10-24</td>\n",
       "      <td>West Ham United FC</td>\n",
       "      <td>Manchester City FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Manchester City FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>48</th>\n",
       "      <td>2020-10-24</td>\n",
       "      <td>Fulham FC</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Crystal Palace FC</td>\n",
       "      <td>Fulham FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>49</th>\n",
       "      <td>2020-10-24</td>\n",
       "      <td>Manchester United FC</td>\n",
       "      <td>Chelsea FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Manchester United FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50</th>\n",
       "      <td>2020-10-24</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Sheffield United FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "      <td>Liverpool FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>51</th>\n",
       "      <td>2020-10-25</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Everton FC</td>\n",
       "      <td>Southampton FC</td>\n",
       "      <td>Everton FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>52</th>\n",
       "      <td>2020-10-25</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "      <td>Newcastle United FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Wolverhampton Wanderers FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>53</th>\n",
       "      <td>2020-10-25</td>\n",
       "      <td>Arsenal FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Leicester City FC</td>\n",
       "      <td>Arsenal FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>54</th>\n",
       "      <td>2020-10-26</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "      <td>West Bromwich Albion FC</td>\n",
       "      <td>Tie</td>\n",
       "      <td>Brighton &amp; Hove Albion FC</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>55</th>\n",
       "      <td>2020-10-26</td>\n",
       "      <td>Burnley FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "      <td>Tottenham Hotspur FC</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "          Day                HomeTeamName                AwayTeamName  \\\n",
       "0  2020-09-12                   Fulham FC                  Arsenal FC   \n",
       "1  2020-09-12           Crystal Palace FC              Southampton FC   \n",
       "2  2020-09-13     West Bromwich Albion FC           Leicester City FC   \n",
       "3  2020-09-13        Tottenham Hotspur FC                  Everton FC   \n",
       "4  2020-09-14         Sheffield United FC  Wolverhampton Wanderers FC   \n",
       "5  2020-09-14   Brighton & Hove Albion FC                  Chelsea FC   \n",
       "6  2020-09-19                  Everton FC     West Bromwich Albion FC   \n",
       "7  2020-09-19             Leeds United FC                   Fulham FC   \n",
       "8  2020-09-19        Manchester United FC           Crystal Palace FC   \n",
       "9  2020-09-19                  Arsenal FC          West Ham United FC   \n",
       "10 2020-09-20              Southampton FC        Tottenham Hotspur FC   \n",
       "11 2020-09-20         Newcastle United FC   Brighton & Hove Albion FC   \n",
       "12 2020-09-20                  Chelsea FC                Liverpool FC   \n",
       "13 2020-09-20           Leicester City FC                  Burnley FC   \n",
       "14 2020-09-21              Aston Villa FC         Sheffield United FC   \n",
       "15 2020-09-21  Wolverhampton Wanderers FC          Manchester City FC   \n",
       "16 2020-09-26   Brighton & Hove Albion FC        Manchester United FC   \n",
       "17 2020-09-26           Crystal Palace FC                  Everton FC   \n",
       "18 2020-09-26     West Bromwich Albion FC                  Chelsea FC   \n",
       "19 2020-09-26                  Burnley FC              Southampton FC   \n",
       "20 2020-09-27         Sheffield United FC             Leeds United FC   \n",
       "21 2020-09-27        Tottenham Hotspur FC         Newcastle United FC   \n",
       "22 2020-09-27          Manchester City FC           Leicester City FC   \n",
       "23 2020-09-27          West Ham United FC  Wolverhampton Wanderers FC   \n",
       "24 2020-09-28                   Fulham FC              Aston Villa FC   \n",
       "25 2020-09-28                Liverpool FC                  Arsenal FC   \n",
       "26 2020-10-03                  Chelsea FC           Crystal Palace FC   \n",
       "27 2020-10-03                  Everton FC   Brighton & Hove Albion FC   \n",
       "28 2020-10-03             Leeds United FC          Manchester City FC   \n",
       "29 2020-10-03         Newcastle United FC                  Burnley FC   \n",
       "30 2020-10-04           Leicester City FC          West Ham United FC   \n",
       "31 2020-10-04              Southampton FC     West Bromwich Albion FC   \n",
       "32 2020-10-04  Wolverhampton Wanderers FC                   Fulham FC   \n",
       "33 2020-10-04                  Arsenal FC         Sheffield United FC   \n",
       "34 2020-10-04        Manchester United FC        Tottenham Hotspur FC   \n",
       "35 2020-10-04              Aston Villa FC                Liverpool FC   \n",
       "36 2020-10-17                  Everton FC                Liverpool FC   \n",
       "37 2020-10-17                  Chelsea FC              Southampton FC   \n",
       "38 2020-10-17          Manchester City FC                  Arsenal FC   \n",
       "39 2020-10-17         Newcastle United FC        Manchester United FC   \n",
       "40 2020-10-18         Sheffield United FC                   Fulham FC   \n",
       "41 2020-10-18           Crystal Palace FC   Brighton & Hove Albion FC   \n",
       "42 2020-10-18        Tottenham Hotspur FC          West Ham United FC   \n",
       "43 2020-10-18           Leicester City FC              Aston Villa FC   \n",
       "44 2020-10-19     West Bromwich Albion FC                  Burnley FC   \n",
       "45 2020-10-19             Leeds United FC  Wolverhampton Wanderers FC   \n",
       "46 2020-10-23              Aston Villa FC             Leeds United FC   \n",
       "47 2020-10-24          West Ham United FC          Manchester City FC   \n",
       "48 2020-10-24                   Fulham FC           Crystal Palace FC   \n",
       "49 2020-10-24        Manchester United FC                  Chelsea FC   \n",
       "50 2020-10-24                Liverpool FC         Sheffield United FC   \n",
       "51 2020-10-25              Southampton FC                  Everton FC   \n",
       "52 2020-10-25  Wolverhampton Wanderers FC         Newcastle United FC   \n",
       "53 2020-10-25                  Arsenal FC           Leicester City FC   \n",
       "54 2020-10-26   Brighton & Hove Albion FC     West Bromwich Albion FC   \n",
       "55 2020-10-26                  Burnley FC        Tottenham Hotspur FC   \n",
       "\n",
       "                        Winner                   Moneyline  \n",
       "0                   Arsenal FC                  Arsenal FC  \n",
       "1            Crystal Palace FC              Southampton FC  \n",
       "2            Leicester City FC           Leicester City FC  \n",
       "3                   Everton FC        Tottenham Hotspur FC  \n",
       "4                          Tie  Wolverhampton Wanderers FC  \n",
       "5                   Chelsea FC                  Chelsea FC  \n",
       "6                   Everton FC                  Everton FC  \n",
       "7                          Tie             Leeds United FC  \n",
       "8            Crystal Palace FC        Manchester United FC  \n",
       "9                          Tie                  Arsenal FC  \n",
       "10        Tottenham Hotspur FC        Tottenham Hotspur FC  \n",
       "11   Brighton & Hove Albion FC         Newcastle United FC  \n",
       "12                Liverpool FC                Liverpool FC  \n",
       "13           Leicester City FC           Leicester City FC  \n",
       "14                         Tie              Aston Villa FC  \n",
       "15          Manchester City FC          Manchester City FC  \n",
       "16        Manchester United FC        Manchester United FC  \n",
       "17                  Everton FC                  Everton FC  \n",
       "18                         Tie                  Chelsea FC  \n",
       "19              Southampton FC              Southampton FC  \n",
       "20             Leeds United FC             Leeds United FC  \n",
       "21                         Tie        Tottenham Hotspur FC  \n",
       "22           Leicester City FC          Manchester City FC  \n",
       "23          West Ham United FC  Wolverhampton Wanderers FC  \n",
       "24              Aston Villa FC              Aston Villa FC  \n",
       "25                Liverpool FC                Liverpool FC  \n",
       "26                  Chelsea FC                  Chelsea FC  \n",
       "27                  Everton FC                  Everton FC  \n",
       "28                         Tie          Manchester City FC  \n",
       "29         Newcastle United FC         Newcastle United FC  \n",
       "30          West Ham United FC           Leicester City FC  \n",
       "31              Southampton FC              Southampton FC  \n",
       "32  Wolverhampton Wanderers FC  Wolverhampton Wanderers FC  \n",
       "33                  Arsenal FC                  Arsenal FC  \n",
       "34        Tottenham Hotspur FC        Manchester United FC  \n",
       "35              Aston Villa FC                Liverpool FC  \n",
       "36                         Tie                Liverpool FC  \n",
       "37                         Tie                  Chelsea FC  \n",
       "38          Manchester City FC          Manchester City FC  \n",
       "39        Manchester United FC        Manchester United FC  \n",
       "40                         Tie         Sheffield United FC  \n",
       "41                         Tie   Brighton & Hove Albion FC  \n",
       "42                         Tie        Tottenham Hotspur FC  \n",
       "43              Aston Villa FC           Leicester City FC  \n",
       "44                         Tie     West Bromwich Albion FC  \n",
       "45  Wolverhampton Wanderers FC             Leeds United FC  \n",
       "46             Leeds United FC              Aston Villa FC  \n",
       "47                         Tie          Manchester City FC  \n",
       "48           Crystal Palace FC                   Fulham FC  \n",
       "49                         Tie        Manchester United FC  \n",
       "50                Liverpool FC                Liverpool FC  \n",
       "51              Southampton FC                  Everton FC  \n",
       "52                         Tie  Wolverhampton Wanderers FC  \n",
       "53           Leicester City FC                  Arsenal FC  \n",
       "54                         Tie   Brighton & Hove Albion FC  \n",
       "55        Tottenham Hotspur FC        Tottenham Hotspur FC  "
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "final_result"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
